In [1]:
import pandas as pd
train_df = pd.read_csv('train.csv',encoding='utf-8')

In [2]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
import nltk
sample = train_df['comment_text'][0]
words = nltk.word_tokenize(sample)
print(words)

['Explanation', 'Why', 'the', 'edits', 'made', 'under', 'my', 'username', 'Hardcore', 'Metallica', 'Fan', 'were', 'reverted', '?', 'They', 'were', "n't", 'vandalisms', ',', 'just', 'closure', 'on', 'some', 'GAs', 'after', 'I', 'voted', 'at', 'New', 'York', 'Dolls', 'FAC', '.', 'And', 'please', 'do', "n't", 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'I', "'m", 'retired', 'now.89.205.38.27']


In [52]:
## Tokens preprocessing
import re, string, unicodedata
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer

""""
import contractions

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)
""""

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def to_lowercase(words):
    return [word.lower() for word in words]

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

stemmer = LancasterStemmer()
def stem_words(words):
    return [stemmer.stem(word) for word in words]

ImportError: No module named contractions

In [ ]:
# class distribution
replace_contractions(sample)

In [42]:
def preprocess(text):
    words = nltk.word_tokenize(text)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = to_lowercase(words)
    words = stem_words(words)
    return words

In [20]:
from nltk.stem import LancasterStemmer, WordNetLemmatizer
stemmer = LancasterStemmer()
print stemmer.stem('curl'),stemmer.stem('curly'),stemmer.stem('curling')
lemmatizer = WordNetLemmatizer()
print lemmatizer.lemmatize('curling','v'),lemmatizer.lemmatize('curl')

curl cur curl
curl curl


In [96]:
def stem(word):
    return stemmer.stem(word)

In [23]:
from collections import Counter
Counter(words)

Counter({'clos': 1,
         'dol': 1,
         'edit': 1,
         'expl': 1,
         'fac': 1,
         'fan': 1,
         'gas': 1,
         'hardc': 1,
         'mad': 1,
         'metallic': 1,
         'new': 1,
         'now892053827': 1,
         'nt': 2,
         'pag': 1,
         'pleas': 1,
         'remov': 1,
         'retir': 1,
         'revert': 1,
         u'sint': 1,
         'talk': 1,
         'templ': 1,
         'usernam': 1,
         'vand': 1,
         'vot': 1,
         'york': 1})

In [54]:
# features collection
import collections
classes = [collections.defaultdict(list),collections.defaultdict(list)]
#class_feature_distr = collections.defaultdict(list)
for ind,row in train_df.iterrows():
    #print row['comment_text']
    words = preprocess(row['comment_text'])
    classes[row['severe_toxic']]['severe_toxic'].extend(words)
    classes[row['obscene']]['obscene'].extend(words)
    classes[row['threat']]['threat'].extend(words)
    classes[row['insult']]['insult'].extend(words)
    classes[row['identity_hate']]['identity_hate'].extend(words)
    

KeyboardInterrupt: 

In [132]:
# class distr
toxic_pos = list(train_df[train_df['toxic']==0].index)
toxic_neg = list(train_df[train_df['toxic']==1].index)
severe_toxic_pos = list(train_df[train_df['severe_toxic']==0].index)
severe_toxic_neg = list(train_df[train_df['severe_toxic']==1].index)
obscene_pos = list(train_df[train_df['obscene']==0].index)
obscene_neg = list(train_df[train_df['obscene']==1].index)
threat_pos = list(train_df[train_df['threat']==0].index)
threat_neg = list(train_df[train_df['threat']==1].index)
insult_pos = list(train_df[train_df['insult']==0].index)
insult_neg = list(train_df[train_df['insult']==1].index)
identity_hate_pos = list(train_df[train_df['identity_hate']==0].index)
identity_hate_neg = list(train_df[train_df['identity_hate']==1].index)

In [49]:
# split train test
import random
def train_test_split(column_name):
    pos_indices = list(train_df[train_df[column_name]==0].index)
    neg_indices = list(train_df[train_df[column_name]==1].index)
    n = int(len(neg_indices)*0.9)
    
    train_indices = pos_indices[:n]+neg_indices[:n]
    test_indices = pos_indices[n:]+neg_indices[n:]

    random.shuffle(train_indices)
    X_train = train_df.loc[train_indices]['comment_text']
    y_train = train_df.loc[train_indices][column_name]
    X_test = train_df.loc[test_indices]['comment_text']
    y_test = train_df.loc[test_indices][column_name]
    return X_train,y_train,X_test,y_test

In [10]:
n = 5914
pos_indices = list(train_df[train_df['obscene']==0].index)
neg_indices = list(train_df[train_df['obscene']==1].index)
train_indices = pos_indices[:n]+neg_indices[:n]
test_indices = pos_indices[n:]+neg_indices[n:]
import random
random.shuffle(train_indices)
X_train = train_df.loc[train_indices]['comment_text']
y_train = train_df.loc[train_indices]['obscene']
X_test = train_df.loc[test_indices]['comment_text']
y_test = train_df.loc[test_indices]['obscene']


In [6]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer


In [72]:
import numpy as np
tr_predicted = text_clf.predict(X_train)
print np.mean(tr_predicted == y_train) 

te_predicted = text_clf.predict(X_test)
print np.mean(te_predicted == y_test) 

0.8960940142035847
0.937147614438586


In [73]:

print confusion_matrix(y_train, tr_predicted)

[[5625  289]
 [ 940 4974]]


In [14]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import numpy as np
def evaluate_classifier(classifier):
    text_clf = Pipeline([
        ('count',CountVectorizer()),
        ('tfidf',TfidfTransformer()),
        ('clf',classifier)
    ])
    
    #tfidf_vect = TfidfVectorizer(preprocessor=stem)
    #X = tfidf_vect.fit_transform(X_train)
    #classifier.fit(X, y_train)
    text_clf.fit(X_train, y_train) 
    tr_predicted = text_clf.predict(X_train)
    print 'Training:'
    print np.mean(tr_predicted == y_train) 
    print roc_auc_score(y_train, tr_predicted)
    print confusion_matrix(y_train, tr_predicted),'\n'
            
    te_predicted = text_clf.predict(X_test)
    print 'Test:'
    print np.mean(te_predicted == y_test) 
    print roc_auc_score(y_test, te_predicted)
    print confusion_matrix(y_test, te_predicted),'\n'  
    return text_clf

In [19]:
#clf = evaluate_classifier(SGDClassifier())
clf_log = evaluate_classifier(SGDClassifier(loss='log'))
clf_mod_huber = evaluate_classifier(SGDClassifier(loss='modified_huber'))

Training:
0.9413256679066622
0.9413256679066622
[[5697  217]
 [ 477 5437]] 

Test:
0.9378650765180077
0.9071508494867241
[[136344   8864]
 [   316   2219]] 

Training:
0.995434562056138
0.995434562056138
[[5875   39]
 [  15 5899]] 

Test:
0.9456624002490812
0.9301095146707594
[[137398   7810]
 [   218   2317]] 



In [22]:
test_comment = 'Yo bitch Ja Rule is more succesful then you will ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da'
clf_mod_huber.predict_proba([test_comment])
clf_mod_huber.predict([test_comment])

array([1], dtype=int64)

In [17]:
from sklearn.svm import LinearSVC,SVC
evaluate_classifier(SVC(kernel='linear',probability=True))

Training:
0.9797937098410551
0.9797937098410551
[[5813  101]
 [ 138 5776]] 

Test:
0.9524715214934041
0.9283410374420935
[[138431   6777]
 [   245   2290]] 



Pipeline(memory=None,
     steps=[('count', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        s...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [30]:
models = []
for i in range(2,8):
    col_name = train_df.columns[i]
    X_train,y_train,X_test,y_test = train_test_split(col_name)
    clf = evaluate_classifier(SGDClassifier(loss='modified_huber'))
    models.append((col_name,clf))

Training:
0.9779542269967305
0.9779542269967304
[[10457   248]
 [  224 10481]] 

Test:
0.9125440609144404
0.8985860235543381
[[122023  11549]
 [   534   4055]] 

Training:
0.9982078853046595
0.9982078853046594
[[1115    1]
 [   3 1113]] 

Test:
0.9155708374910225
0.9097864350935118
[[143622  13238]
 [    46    433]] 

Training:
0.9955191072032465
0.9955191072032465
[[5885   29]
 [  24 5890]] 

Test:
0.9567221458884685
0.9303096574136949
[[139060   6148]
 [   246   2289]] 

Training:
1.0
1.0
[[334   0]
 [  0 334]] 

Test:
0.8893853482942424
0.8926064165391149
[[141197  17562]
 [    15    129]] 

Training:
0.99383275893343
0.99383275893343
[[5462   51]
 [  17 5496]] 

Test:
0.9277121411020229
0.9137472104067355
[[135681  10500]
 [   238   2126]] 

Training:
0.9989827060020345
0.9989827060020346
[[983   0]
 [  2 981]] 

Test:
0.8819263348244027
0.8888119299950826
[[138618  18565]
 [    44    378]] 



In [36]:
test_df = pd.read_csv('test.csv',encoding='utf-8')
columns = ['id']+list(train_df.columns[2:8])
results_df = pd.DataFrame(columns=columns)
results_df['id'] = test_df['id']
    

In [31]:
for colname,model in models:
    res = model.predict_proba(test_df['comment_text'])
    results_df[colname] = res

In [32]:
results_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,0000247867823ef7,1.000000,0.823851,1.000000,1.0,1.000000,1.000000
2,00013b17ad220c46,1.000000,1.000000,0.891873,1.0,0.743326,1.000000
3,00017563c3f7919a,1.000000,1.000000,1.000000,1.0,1.000000,0.949218
4,00017695ad8997eb,0.765029,1.000000,0.908508,1.0,0.982901,1.000000


In [33]:
results_df.to_csv('test_predicted.csv')

In [56]:
all_text = train_df['comment_text']
vectoriser = TfidfVectorizer(sublinear_tf=True,
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=15000)
vectoriser.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=15000, min_df=1,
        ngram_range=(1, 2), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [41]:
train_df.columns

Index([u'id', u'comment_text', u'toxic', u'severe_toxic', u'obscene',
       u'threat', u'insult', u'identity_hate'],
      dtype='object')

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
colnames = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for colname in colnames:
    X_train,train_target,X_test,y_test = train_test_split(colname)
    #X_train,train_target = train_df['comment_text'], train_df[colname]
    
    train_features = vectoriser.transform(X_train)
    classifier = LogisticRegression(solver='sag')#SGDClassifier(loss='log')
    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    print colname, cv_score
    classifier.fit(train_features,train_target)
    test_features = vectoriser.transform(X_test)
    te_predicted = classifier.predict(test_features)
    print 'test score accuracy', np.mean(te_predicted == y_test),roc_auc_score(y_test, te_predicted),'\n'
    

toxic 0.9603824160521857
test score accuracy 0.9334307763380111 0.9007630246835402 

severe_toxic 0.982333297970963
test score accuracy 0.9688642701705796 0.915736492356635 

obscene 0.9805624341456918
test score accuracy 0.9714123424977315 0.9179753219005331 

threat 0.973149138328577
test score accuracy 0.9708715842002129 0.9021232633527246 

insult 0.969496111557146
test score accuracy 0.9526455881644921 0.9074082214511543 

identity_hate 0.9682494511404242
test score accuracy 0.9532548410308005 0.9092910775674132 



In [55]:
for colname in colnames:
    X_train,train_target = train_df['comment_text'], train_df[colname]
    
    train_features = vectoriser.transform(X_train)
    classifier = LogisticRegression(solver='sag')#SGDClassifier(loss='log')
    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    print colname, cv_score

toxic 0.9693287995396691
severe_toxic 0.9860163108666002
obscene 0.985220193390691
threat 0.9823253412465484
insult 0.976192055968359
identity_hate 0.9743973759002674


In [ ]:
toxic 0.9694377089374315
severe_toxic 0.9835987922361634
obscene 0.9825913759390813
threat 0.9827427709355318
insult 0.9749755607373317
identity_hate 0.9722185220302423


toxic 0.9585138739296148
test score accuracy 0.9240550426754921 0.9024795291221083
severe_toxic 0.9794910664963711
test score accuracy 0.9624188741616199 0.9125105004439732
obscene 0.9744300964732768
test score accuracy 0.9611742621031705 0.9187084629344887
threat 0.9691597213480713
test score accuracy 0.949638021309172 0.8915032700335513
insult 0.9663699077331698
test score accuracy 0.9378787149312553 0.9062951507844668
identity_hate 0.9585100649984408
test score accuracy 0.9355335799749114 0.9145941347858549